In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.cross_validation import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import cohen_kappa_score
from scipy.special import expit
from scipy.special import logit
from sklearn.cross_validation import train_test_split
%matplotlib inline

time: 480 ms


In [41]:
#Turn off SettingWithCopyWarning()
pd.options.mode.chained_assignment = None

time: 1.6 ms


In [2]:
!pwd

/Users/Jake/Documents/Relief-Fatigue/Code
time: 122 ms


In [2]:
df = pd.read_csv('../Data/pitch_swing.csv', index_col=0)
inds = np.random.choice(df.shape[0], df.shape[0], replace=False)
df = df.iloc[inds].copy()

time: 5.64 s


In [3]:
list(df.pitch_type.unique())

['FF', 'SL', 'SI', 'FC', 'CH', 'FT', 'CU', 'FS']

time: 32.9 ms


In [360]:
df.columns

Index(['gameday_link', 'batter', 'pitcher', 'num', 'event', 'inning_side',
       'inning', 'batter_name', 'pitcher_name', 'date', 'o', 'des', 'type',
       'x', 'y', 'start_speed', 'end_speed', 'sz_top', 'sz_bot', 'pfx_x',
       'pfx_z', 'px', 'pz', 'x0', 'y0', 'z0', 'vx0', 'vy0', 'vz0', 'ax', 'ay',
       'az', 'break_y', 'break_angle', 'break_length', 'pitch_type',
       'spin_dir', 'spin_rate', 'count', 'zone', 'sv_id', 'reg_season',
       'post_season', 'year', 'b', 's', 'avg_fast_speed.x', 'swinging',
       'whiff', 'avg_fast_speed.y', 'in_zone'],
      dtype='object')

time: 5.75 ms


# Get best leaf for each pitch_type

**Also return table of results**

In [49]:
#k-fold cross validation
def get_best_leaf(df,pitch_list,leaf_list,covariates,folds = 10):

    fits={}
    imps = {}

    for pitch in pitch_list:
        df_p = df[df.pitch_type==pitch]
        fits[pitch] ={}
        Y = df_p.whiff
        X = df_p[covariates]
        avg_scores = []; avg_kappas = [];
        for leaf in leaf_list:
            scores = []; kappas = []; 
            clf = RandomForestClassifier(min_samples_leaf=leaf)

            for train, test in KFold(Y.shape[0], n_folds=folds):
                #Fit model
                clf.fit(X.iloc[train], Y.iloc[train])

                #Judge the model
                scores.append(clf.score(X.iloc[test], Y.iloc[test]))
                preds = clf.predict(X.iloc[test])
                kappas.append(cohen_kappa_score(preds, Y.iloc[test]))

            #print('\nNumber of Leaves: {}'.format(leaf))
            #print('Avg Score: {0:.3f}'.format(np.mean(scores)))
            #print('Sd Score: {0:.2f}'.format(np.std(scores)))
            #print('Avg Kappa: {0:.3f}'.format(np.mean(kappas)))
            avg_scores.append(round(np.mean(scores),3))
            avg_kappas.append(round(np.mean(kappas),3))

        best_k = np.argmax(avg_kappas)
        best_s = np.argmax(avg_scores)
        print('\nPitch Type: {}'.format(pitch))
        print("Percent Whiffs: ", round(1-Y.mean(),3))
        print('Leaves/Score/Kappa for best:')
        print('Score: ',leaf_list[best_s],"/",avg_scores[best_s],"/",avg_kappas[best_s])
        print('Kappa: ',leaf_list[best_k],"/",avg_scores[best_k],"/",avg_kappas[best_k])

        fits[pitch]['perc_whiff'] = round(1-Y.mean(),3)
        fits[pitch]['score'] = avg_scores[best_s]
        fits[pitch]['num_pitches'] = int(len(Y))
        fits[pitch]['leaf'] = int(leaf_list[best_s])
        fits[pitch]['kappa'] = avg_kappas[best_s]

    return(fits)



time: 74.5 ms


In [50]:
leafs = [25,50,100,200,500];
pitch_types = list(df.pitch_type.unique())
covars = ['start_speed','end_speed','pfx_x','pfx_z','vx0','vy0','vz0','ax','ay','az','break_y',
         'break_angle','spin_dir','spin_rate']
best = get_best_leaf(df,pitch_types,leafs,covars)


Pitch Type: FT
Percent Whiffs:  0.799
Leaves/Score/Kappa for best:
Score:  25 / 0.802 / 0.05
Kappa:  25 / 0.802 / 0.05

Pitch Type: CH
Percent Whiffs:  0.536
Leaves/Score/Kappa for best:
Score:  50 / 0.631 / 0.25
Kappa:  50 / 0.631 / 0.25

Pitch Type: SL
Percent Whiffs:  0.545
Leaves/Score/Kappa for best:
Score:  25 / 0.683 / 0.351
Kappa:  25 / 0.683 / 0.351

Pitch Type: FF
Percent Whiffs:  0.759
Leaves/Score/Kappa for best:
Score:  25 / 0.76 / 0.056
Kappa:  25 / 0.76 / 0.056

Pitch Type: FC
Percent Whiffs:  0.695
Leaves/Score/Kappa for best:
Score:  25 / 0.715 / 0.142
Kappa:  25 / 0.715 / 0.142

Pitch Type: SI
Percent Whiffs:  0.798
Leaves/Score/Kappa for best:
Score:  25 / 0.802 / 0.058
Kappa:  25 / 0.802 / 0.058

Pitch Type: FS
Percent Whiffs:  0.553
Leaves/Score/Kappa for best:
Score:  25 / 0.664 / 0.31
Kappa:  25 / 0.664 / 0.31

Pitch Type: CU
Percent Whiffs:  0.533
Leaves/Score/Kappa for best:
Score:  25 / 0.707 / 0.407
Kappa:  25 / 0.707 / 0.407
time: 8min 57s


# Get Predicted Means Table

In [ ]:
def get_pred_means_table(df,covariates,leaf, subset = None):
    df_p_train, df_p_test = train_test_split(df, test_size=0.5, random_state=42)

    #Test and train X datasets
    Y_train = df_p_train.whiff
    Y_test = df_p_test.whiff
    X_train = df_p_train[covariates]

    X_test = df_p_test[covariates]


    X_train = pd.concat([pd.get_dummies(X_train[col]).ix[:, :-1] 
                   if X_train[col].dtype == object or hasattr(X_train[col], 'cat')
                   else X_train[col]
                   for col in X_train.columns], axis=1)

    X_test = pd.concat([pd.get_dummies(X_test[col]).ix[:, :-1] 
                   if X_test[col].dtype == object or hasattr(X_test[col], 'cat')
                   else X_test[col]
                   for col in X_test.columns], axis=1)

    #Fit the Random Forest with the best leaf
    clf = RandomForestClassifier(min_samples_leaf=leaf)
    clf.fit(X_train,Y_train)

    print(1-Y_test.mean())
    print(clf.score(X_test,Y_test))
    #Predict probabilities
    pred_probs = clf.predict_proba(X_test)[:,1] #predicted probability of strike

    #Take the logit
    pred_stuff = logit(pred_probs)

    pred_mat = df_p_test[["pitcher_name"]].copy()
    pred_mat['pred_stuff'] = pred_stuff
    
    #Get rid of people with predicted probability 0
    pred_mat['pred_stuff'] = pred_mat['pred_stuff'][pred_mat['pred_stuff']>-math.inf]
    
    #mean/sd for standardization
    overall_mean = pred_mat.pred_stuff.mean()
    overall_sd = pred_mat.pred_stuff.std()
    
    #Standardize
    pred_mat['pred_z'] = (pred_mat['pred_stuff'] - overall_mean)/overall_sd
    
    #group by pitcher name
    #average the stuff
    group_pred = pred_mat.groupby('pitcher_name')

    mapped_funs = {'pred_z': {'count','mean'}}
    pred_means = group_pred.agg(mapped_funs)

    #Only show me players with more than 100 of that pitch
    if(subset is not None):
        pred_means = pred_means[pred_means[('pred_z','count')]>subset]
    
    pred_means = pred_means.sort_values(('pred_z','mean'),ascending = False)

    return(pred_means)

# Train Model based on best leaf, attach to other dset

** Need to train model on 2012/2016 swings, but attach to 2013-2015 all pitches**

In [68]:
def add_stuff(train_dset, pred_dset, best_fits, covariates):
    
    #Train all the pitches, save the clfs
    all_pitches = train_dset.pitch_type.unique()
    if(len(all_pitches)!=len(best_fits)):
        print('Error: Num pitch types in fits dictionary not same as num pitch types in dset')
        return
    
    clf_dict = {}
    for i in range(len(all_pitches)):
        pitch = all_pitches[i]
        
        if(pitch not in best_fits.keys()):
            print(pitch+' not in fits dictionary')
            return
        
        train_df = train_dset[train_dset.pitch_type==pitch]
        X_train = train_df[covariates]

        X_train = pd.concat([pd.get_dummies(X_train[col]).ix[:, :-1] 
                       if X_train[col].dtype == object or hasattr(X_train[col], 'cat')
                       else X_train[col]
                       for col in X_train.columns], axis=1)
        
        Y_train = train_df.whiff
        
        leaf = best_fits[pitch]['leaf']
        clf =  RandomForestClassifier(min_samples_leaf=leaf)
        clf.fit(X_train,Y_train)
        
        ##Now, predict on new data
        pred_df = pred_dset[pred_dset.pitch_type==pitch].copy()
        
        X_pred = pred_df[covariates]
        X_pred = pd.concat([pd.get_dummies(X_pred[col]).ix[:, :-1] 
                       if X_pred[col].dtype == object or hasattr(X_pred[col], 'cat')
                       else X_pred[col]
                       for col in X_pred.columns], axis=1)
        
        #Predict probabilities
        pred_probs = clf.predict_proba(X_pred)[:,1] #predicted probability of strike

        #Take the logit
        pred_stuff = logit(pred_probs)
        
        overall_mean = pred_stuff.mean()
        overall_sd = pred_stuff.std()
        
        pred_z = (pred_stuff - overall_mean)/overall_sd
        
        pred_df['z_stuff'] = pred_z
        
        if(i==0):
            out_mat = pred_df.copy()
        else:
            out_mat = out_mat.append(pred_df)
        
    
    if(out_mat.shape[0]!= pred_dset.shape[0]):
        print("You got more than zero problems big fella")
        
    return(out_mat)

    

time: 141 ms


In [52]:
final_train = df[(df.year==2026) | (df.year==2012)]
final_pred = df[(df.year>=2013)&(df.year<=2015)]

time: 288 ms


In [69]:
jags_dset = add_stuff(final_train, final_pred,best,covars,)

time: 5.44 s


In [72]:
jags_group = jags_dset.groupby('pitcher_name')

mapped_funs = {'z_stuff': {'count','mean'}}
z_stuff = jags_group.agg(mapped_funs)
z_stuff.sort_values(('z_stuff','mean'),ascending=False)

z_stuff      
                         mean count
pitcher_name                       
Zachary Britton      1.442092   646
Aroldis Chapman      1.102518  1267
Jacob McGee          1.015731  1166
Scott Rice           0.929227   315
Blake Treinen        0.920125   318
Brayan Villarreal    0.847694    37
Craig Kimbrel        0.831652  1198
Enny Romero          0.829765   190
Jose Dominguez       0.735225    35
Carter Capps         0.726589   650
Jeff Beliveau        0.726322    61
Ben Rowen            0.713083    32
Rhiner Cruz          0.709072    82
Trevor Gott          0.702214   182
Kyuji Fujikawa       0.665292    93
Brian Ellington      0.641354   141
Michael Kohn         0.637815   442
Darin Downs          0.629135   323
Jeurys Familia       0.628811   869
Kyle Crockett        0.626520   220
Ernesto Frieri       0.590023   944
Heath Hembree        0.581815   145
Jose Ramirez         0.574212   100
Andrew Bailey        0.547582   173
Austin Adams         0.545419   144
Cameron Bedrosian    0.534837   283
B.J. Rosenberg       0.512843    49
Gus Schlosser        0.499536    74
Michael Foltynewicz  0.490488    44
Joshua Fields        0.478141   918
...                       ...   ...
Pat Neshek          -0.495140   943
Donn Roach          -0.505128   115
Eric Surkamp        -0.509439    35
Brandon Lyon        -0.510155   125
Zeke Spruill        -0.511600    46
Jose Valverde       -0.515800   203
Tim Stauffer        -0.529176    51
Edgmer Escalona     -0.543342   291
Paul Clemens        -0.546011    82
Derek Lowe          -0.562304    48
Eury De La Rosa     -0.563159   117
Shawn Camp          -0.566803   117
Jean Machi          -0.570259   750
Michael Tonkin      -0.598006   214
Sugar Ray Marimon   -0.600120    83
Miguel Socolovich   -0.607890   161
Sam LeCure          -0.612811   537
Adam Achter         -0.618581    39
Joe Savery          -0.619563    69
Daniel Otero        -0.652545   764
Wei-Chung Wang      -0.663432    85
Drew VerHagen       -0.692595    73
Mike Adams          -0.694611    74
Thad Weber          -0.696752    58
Burke Badenhop      -0.705492   771
LaTroy Hawkins      -0.726014   783
Kenneth Roberts     -0.736238    42
Collin Balester     -0.789233    55
Blake Parker        -0.833456   368
Kevin Gregg         -0.896854   354

[475 rows x 2 columns]

time: 63 ms


In [64]:
final_pred.shape

(193706, 51)

time: 3.18 ms


**This is where the money is made**